# Use `geeet.common.relative_humidity` to calcualte relative humidity 

In this notebook we calculate relative humidity for the ECMWF land-hourly dataset using the `temperature_2m`, `dewpoint_temperature_2m`, and `surface_pressure` bands. The calculations are based on equations 7.4, 7.5, 7.91, and 7.93 (for water only - not ice) in [PART IV: PHYSICAL PROCESSED of the ECMWF IFS documentation](https://www.ecmwf.int/sites/default/files/elibrary/2016/17117-part-iv-physical-processes.pdf#subsection.7.2.1). 

In [1]:
import geemap
geemap.ee_initialize()
import ee

date_start = ee.Date('2019-06-01')
date_end = date_start.advance(1, 'month')
lonlat = [0, 0]
geom = ee.Geometry.Point(lonlat[0], lonlat[1])


## Relative humidity from another dataset (to compare)
RH_collection = ee.ImageCollection('NOAA/GFS0P25').select('relative_humidity_2m_above_ground') # in % (0-100)

## ECMWF collection 
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands)
bands_to_keep = ee.List(['temperature_2m', 'dewpoint_temperature_2m', 'surface_pressure'])
bands_to_rename = ee.List(['temperature', 'dewpoint_temperature', 'surface_pressure']) # see geeet.common.relative_humidity 
# and related functions. 
Meteo_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
    .filterDate(date_start, date_end) \
        .filterBounds(geom) \
            .select(bands_to_keep, bands_to_rename)

from geeet.common import relative_humidity

Meteo_collection = Meteo_collection.map(relative_humidity)

Map = geemap.Map(center=[lonlat[1], lonlat[0]], zoom=3)
import geemap.colormaps as cm
pal_viridis = cm.palettes.viridis

test_img = Meteo_collection.filter(ee.Filter.eq("system:index", "20190602T12"))
compare_img = RH_collection.filter(ee.Filter.eq("system:index", "2019060212F000")).first()

Map.addLayer(compare_img, {'min':0, 'max':100, 'palette': pal_viridis}, 'Reference RH')
Map.addLayer(test_img.select('relative_humidity'), {'min':0, 'max':100, 'palette': pal_viridis}, 'Computed RH')

Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…